In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import re

In [12]:
driver = webdriver.Chrome()
driver.get("https://www.vlance.vn/#")

login_button = driver.find_element(By.ID, "btn-login")
login_button.click()
time.sleep(5)

username = driver.find_element(By.ID, "login_username_header")
password = driver.find_element(By.ID, "login_password_header")
username.send_keys("qn9965mh@gmail.com")
password.send_keys("123456asd*")

print('Solve captcha manually...............')
time.sleep(30)

sumbit_button = driver.find_element(By.ID, "btn-submit-login-header")
sumbit_button.click()

Solve captcha manually...............


In [6]:
df = pd.read_csv('../../../data/raw/job_posts_219.csv', encoding='utf-8')


In [7]:
# Ensure that the Applicants column is a list-like structure
df = df.dropna(subset=['Applicants'])
df['Applicants'] = df['Applicants'].apply(lambda x: x.replace('[', '').replace(']', ''))
df['Applicants'] = df['Applicants'].apply(lambda x: x.split(','))

# Apply the explode method only if the values are lists
applicants = df.explode('Applicants')
applicants.head()

,Type,Title,Services,Skills,Description,In4_project,In4_employment,Num_applicants,Applicants,Duration,Price,Link
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,'/freelancer/hang-nguyen-747',\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,'/freelancer/chi-chi-109',\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,'/freelancer/minh-tri-truong-2',\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,'/freelancer/cuong-market10',\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,'/freelancer/ngoc-duy-nguyen-1',\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem


In [8]:
applicants['Applicants'] = applicants['Applicants'].apply(lambda x: x.replace("'", '') if isinstance(x, str) else x)
applicants = applicants.reset_index(drop=True)
applicants.head()

,Type,Title,Services,Skills,Description,In4_project,In4_employment,Num_applicants,Applicants,Duration,Price,Link
0,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,/freelancer/hang-nguyen-747,\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
1,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,/freelancer/chi-chi-109,\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
2,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,/freelancer/minh-tri-truong-2,\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
3,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,/freelancer/cuong-market10,\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem
4,Đăng việc,\nKiểm thử phần mềm Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nPhân tích , \nTesti...",\n\nKhi test 1 sản phẩm cần: \nLập kế hoạch ki...,\n\nID dự án\n71013\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,/freelancer/ngoc-duy-nguyen-1,\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-lam/kiem-thu-phan-mem


In [10]:
ID_project = re.search(r'ID dự án\n\d+', applicants['In4_project'][0].strip()).group(0)
ID_project

'ID dự án\n71013'

In [14]:
count = 0
df_freelancer = pd.DataFrame(columns=['id', 'id_project', 'title', 'region', 'overview', 'services', 'summary_profile', 'summary_working', 'link'])
ID = []
ID_project = []
Title = []
Skills = []
Region = []
Overview = []
Services = []
Summary_profile = []
Summary_working = []
Link = []



for i in range(0, applicants.shape[0]):
    ID_project.append(re.search(r'ID dự án\n\d+', applicants['In4_project'][i].strip()).group(0))

    url = "https://www.vlance.vn"+ applicants['Applicants'][i].strip()
    Link.append(url)
    print(url)

    driver.get(url)
    count += 1

    time.sleep(10)
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.id_profile"))
        )
    except Exception as e:
        print("Error: ", e)
        driver.quit()
    
    try:
        see_more_skill = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "see-more-skill"))
        )
        see_more_skill.click()
        print("Clicked on 'see-more-skill'")
    except Exception as e:
        print("Element 'see-more-skill' not found or not clickable:", e)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    id = soup.find('div', attrs={'class':"id_profile"}).text
    ID.append(id)
    # name = soup.find('span', attrs={'itemprop':"name"}).text
    title = soup.find('span', attrs={'itemprop':"title editable tf300"})
    if title:
        Title.append(title.text)
    else:
        Title.append(np.nan)

    skills = soup.find('div', attrs={'class':"skills"}).text
    Skills.append(skills)

    region = soup.find('div', attrs={'class':"regional-price"})
    if region:
        Region.append(region.text)
    else:
        Region.append(np.nan)

    overview = soup.find('div', attrs={'class':"overview overview-mobile"})
    if overview:
        Overview.append(overview.text)
    else:
        Overview.append(np.nan)

    services = soup.find('div', attrs={'class':"service-freelancer"})
    if services:
        Services.append(services.text)
    else:
        Services.append(np.nan)

    summary_profile = soup.find('div', attrs={'class':"summary-profile profile-rate summary-title summary-block"}).text
    summary_working = soup.find('div', attrs={'class':"summary-profile profile-rate summary-working"}).text

    if count == 1:
        df_freelancer = pd.DataFrame({'id': ID, 'id_project': ID_project, 'title': Title, 'region': Region, 
                                      'overview': Overview, 'services': Services, 
                                      'summary_profile': summary_profile, 'summary_working': summary_working,
                                      'link': Link})
        df.head()
        
        df_freelancer.to_csv(f'../../../data/raw/freelancer_{i}.csv', index=False)
        df_freelancer = pd.DataFrame(columns=['id', 'id_project', 'title', 'region', 'overview', 'services', 'summary_profile', 'summary_working', 'link'])
        ID = []
        ID_project = []
        Title = []
        Skills = []
        Region = []
        Overview = []
        Services = []
        Summary_profile = []
        Summary_working = []
        Link = []


https://www.vlance.vn/freelancer/hang-nguyen-747
Element 'see-more-skill' not found or not clickable: Message: 
Stacktrace:
#0 0x63462109cb9a <unknown>
#1 0x634620d82670 <unknown>
#2 0x634620dd1c48 <unknown>
#3 0x634620dd1ee1 <unknown>
#4 0x634620e18924 <unknown>
#5 0x634620df6bad <unknown>
#6 0x634620e15dd6 <unknown>
#7 0x634620df6923 <unknown>
#8 0x634620dc46e7 <unknown>
#9 0x634620dc56de <unknown>
#10 0x63462106666b <unknown>
#11 0x63462106a611 <unknown>
#12 0x6346210524e5 <unknown>
#13 0x63462106b192 <unknown>
#14 0x6346210376ef <unknown>
#15 0x63462108b9d8 <unknown>
#16 0x63462108bba7 <unknown>
#17 0x63462109b9ec <unknown>
#18 0x7f3934a94ac3 <unknown>

https://www.vlance.vn/freelancer/chi-chi-109
Element 'see-more-skill' not found or not clickable: Message: 
Stacktrace:
#0 0x63462109cb9a <unknown>
#1 0x634620d82670 <unknown>
#2 0x634620dd1c48 <unknown>
#3 0x634620dd1ee1 <unknown>
#4 0x634620e18924 <unknown>
#5 0x634620df6bad <unknown>
#6 0x634620e15dd6 <unknown>
#7 0x634620df6923 

KeyboardInterrupt: 